In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
from keras.engine import InputSpec, Layer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, BatchNormalization, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras.optimizers import Adam

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

import numpy as np


Using TensorFlow backend.


In [2]:
import dataloader
binary_train, binary_dev, binary_test = dataloader.GetDataSet(binary_label=True)

train, dev, test = dataloader.GetDataSet(binary_label=False)

In [4]:
full_text = list(train['review'].values)
full_text.extend(list(dev['review'].values))
full_text.extend(list(test['review'].values))

array(['For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.',
       'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily 

In [4]:
tk = Tokenizer(lower = True, filters='')
max_len = 300
tk.fit_on_texts(full_text)
train_tokenized = tk.texts_to_sequences(np.array(list(binary_train['review'].values)))
dev_tokenized = tk.texts_to_sequences(binary_dev['review'].values)
test_tokenized = tk.texts_to_sequences(binary_test['review'].values)
X_train = pad_sequences(train_tokenized, maxlen = max_len)
X_validation = pad_sequences(dev_tokenized, maxlen=max_len)
X_test = pad_sequences(test_tokenized, maxlen=max_len)

In [5]:
Y_train = binary_train['sentiment'].values
Y_dev = binary_dev['sentiment'].values
Y_test = binary_test['sentiment'].values

In [6]:
embedding_path = "./pre-trained/crawl-300d-2M.vec"
EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

embed_size = 300
max_features = 30000

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
def generate_embedding_matrix(embedding_path):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))
    word_index = tk.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_features: 
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_matrix = generate_embedding_matrix(embedding_path)

def pretrained_embedding_layer(embedding_matrix):
    vocab_len, emb_dim = embedding_matrix.shape
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([embedding_matrix])
    return embedding_layer

In [12]:
def build_model(embedding_matrix, X_train, Y_ohe, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "best_test_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
    x1 = SpatialDropout1D(spatial_dr)(embeddings)

    x_gru = Bidirectional(CuDNNGRU(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool3_gru = GlobalAveragePooling1D()(x3)
    max_pool3_gru = GlobalMaxPooling1D()(x3)
    
    x_lstm = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool3_lstm = GlobalAveragePooling1D()(x3)
    max_pool3_lstm = GlobalMaxPooling1D()(x3)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                    avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
    x = Dense(8, activation = "softmax")(x)
    model = Model(inputs = inp, outputs = x)
    
    model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_ohe, batch_size = 128, epochs = 20, validation_data=(X_validation, label_y), 
                        verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model, history


def simple_model(embedding_matrix, X_train, Y_ohe, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "best_model12.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                 save_best_only = True, mode = "min")
    #early stop
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
   
   # adding drop out for regularization
    X = Bidirectional(CuDNNLSTM(32, return_sequences = True))(embeddings)
   # Add dropout with a probability of 0.5
    X = GlobalMaxPool1D()(X)
   # Propagate X trough another LSTM layer with 128-dimensional hidden state
   # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Dense(20, activation="relu")(X)
   # Add dropout with a probability of 0.5
    X = Dropout(0.05)(X)
   # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(8)(X)
   # Add a softmax activation
    X = Activation('softmax')(X)
   
    model = Model(inputs = inp, outputs = X)
    #return model
    model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    print(X_train.shape)
    print(y_ohe.shape)
    history = model.fit(X_train, y_ohe, batch_size = 32, epochs = 20, validation_data=(X_validation, label_y), 
                       verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

def cnn_model(embedding_matrix, X_train, Y_ohe, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "best_model12.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                 save_best_only = True, mode = "min")
    #early stop
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
   
   # adding drop out for regularization
    X = Conv1D(128, 5, activation='relu')(embeddings)
   # Add dropout with a probability of 0.5
    X = MaxPooling1D(5)(X)
    X = Conv1D(128, 5, activation='relu')(X)
    X = MaxPooling1D(5)(X)
    X = Conv1D(128, 5, activation='relu')(X)
    X = MaxPooling1D(7)(X)
    X = Flatten()(X)
    X = Dense(128, activation='relu')(X)
   # Add a softmax activation
    X = Dense(8, activation='softmax')(X)
   
    model = Model(inputs = inp, outputs = X)
    #return model
    model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy", metrics.categorical_accuracy])
    print(X_train.shape)
    print(y_ohe.shape)
    history = model.fit(X_train, y_ohe, batch_size = 32, epochs = 20, validation_split=0.2, 
                       verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [12]:
def build_model_binary(embedding_matrix, X_train, Y_train, max_len = 300, validation_data=None, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "best_test_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
    x1 = SpatialDropout1D(spatial_dr)(embeddings)

    x_gru = Bidirectional(CuDNNGRU(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool3_gru = GlobalAveragePooling1D()(x3)
    max_pool3_gru = GlobalMaxPooling1D()(x3)
    
    x_lstm = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool3_lstm = GlobalAveragePooling1D()(x3)
    max_pool3_lstm = GlobalMaxPooling1D()(x3)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                    avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
    x = Dense(1, activation = "softmax")(x)
    model = Model(inputs = inp, outputs = x)
    
    model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    if validation_data:
        history = model.fit(X_train, Y_train, batch_size = 128, epochs = 20, validation_data=validation_data, verbose = 1, callbacks = [check_point, early_stop])
    else:
        history = model.fit(X_train, Y_train, batch_size = 128, epochs = 20, validation_split=0.2, verbose = 1, callbacks = [check_point, early_stop])
    
    model = load_model(file_path)
    return model, history


def simple_model_binary(embedding_matrix, X_train, Y_train, max_len = 300, validation_data=None, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "best_model12.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                 save_best_only = True, mode = "min")
    #early stop
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
   
   # adding drop out for regularization
    X = Bidirectional(CuDNNLSTM(32, return_sequences = True))(embeddings)
   # Add dropout with a probability of 0.5
    X = GlobalMaxPool1D()(X)
   # Propagate X trough another LSTM layer with 128-dimensional hidden state
   # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Dense(20, activation="relu")(X)
   # Add dropout with a probability of 0.5
    X = Dropout(0.05)(X)
   # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(1)(X)
   # Add a softmax activation
    X = Activation('sigmoid')(X)
   
    model = Model(inputs = inp, outputs = X)
    #return model
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    if validation_data:
        history = model.fit(X_train, Y_train, batch_size = 32, epochs = 20, validation_data=(X_validation, label_y), 
                            verbose = 1, callbacks = [check_point, early_stop])
    else:
        history = model.fit(X_train, Y_train, batch_size = 32, epochs = 20, validation_split=0.2, 
                            verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

def cnn_model_binary(embedding_matrix, X_train, Y_train, max_len = 300, validation_data=None, lr=0.0, lr_d=0.0):
    file_path = "best_model12.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                 save_best_only = True, mode = "min")
    #early stop
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    inp = Input(shape = (max_len,))
    embedding_layer = pretrained_embedding_layer(embedding_matrix)
    embeddings = embedding_layer(inp)
   
    X = Conv1D(128, 5, activation='relu')(embeddings)
    print(X)
    X = MaxPooling1D(5)(X)
    X = Conv1D(128, 5, activation='relu')(X)
    X = MaxPooling1D(5)(X)
    X = Conv1D(128, 5, activation='relu')(X)
    X = MaxPooling1D(7)(X)
    #X = Flatten()(X)
    #X = Dense(64, activation='relu')(X)
    X = Dense(2, activation='sigmoid')(X)
   
    model = Model(inputs = inp, outputs = X)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 32, epochs = 20, validation_split=0.2, 
                       verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [14]:
cnn_model_b = cnn_model_binary(embedding_matrix, X_train, y_ohe, max_len)

ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[node random_uniform_3/min (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4139) ]]

Caused by op 'random_uniform_3/min', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 427, in 

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(Y_train.reshape(-1, 1))
label_y = ohe.fit_transform(Y_dev.reshape(-1, 1))
test_y = ohe.fit_transform(Y_test.reshape(-1, 1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [41]:
X_train.shape

(25000, 300)

In [42]:
embedding_matrix.shape

(30001, 300)